In [19]:
from core.utils import Tibanna
from core import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
    

In [21]:
import time
from tasks import run_fastqc
from invoke import run

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

add_wfr = True

# url for hic exps
fastq_url = "search/?type=FileFastq&quality_metric.uuid=No+value" + \
            "&status=released&status=released%20to%20project&status=uploaded" + \
            "&limit=all" + \
            "&q=date_created%3A%3E%3D2016-09-01"

fastq_files = ff_utils.get_metadata(fastq_url , connection=ff)['@graph']


counter = 0
completed = 0
print len(fastq_files)

okay = 0
non = 0
running = 0
for a_fastq in fastq_files:
    report = get_wfr_out(a_fastq['accession'], 'fastqc-0-11-4-1/1', None, ff)
    if report.startswith('no') or not report:
        non += 1
        if add_wfr:
            print 'fastqc running for', a_fastq['accession']
            code_qc= "invoke run_fastqc " + env + " " + a_fastq['display_title'] + " " + a_fastq['uuid']
            run(code_qc)
        else:
            print 'fastqc run missing for', a_fastq['accession']   
    elif report == 'running':
        running += 1
        print(a_fastq['accession'], 'still running')
    else:
        okay += 1
        
print("okay, not okay, running")    
print(okay, non, running)


1
fastqc running for 4DNFIO67AFHV
about to start run fastqc-0-11-4-1_4DNFIO67AFHV.fastq.gzab2f3bad-9f1e-4b28-b410-2c6de11f7159
response from aws was: 
 {u'startDate': datetime.datetime(2018, 5, 4, 11, 12, 39, 748000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '95df35e2-4fad-11e8-af5d-65911bc23b51', 'HTTPHeaders': {'x-amzn-requestid': '95df35e2-4fad-11e8-af5d-65911bc23b51', 'content-length': '182', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIO67AFHV.fastq.gzab2f3bad-9f1e-4b28-b410-2c6de11f7159'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIO67AFHV.fastq.gzab2f3bad-9f1e-4b28-b410-2c6de11f7159
okay, not okay, running
(0, 1, 0)


In [25]:
# Release fastq qc

fastq_url = '/search/?status=released&status=released%20to%20project&status=archived&type=FileFastq'
fastq_files = ff_utils.get_metadata(fastq_url , connection=ff)['@graph']
fastq_files = [i for i in fastq_files if i.get('quality_metric')]
print len(fastq_files)
success = 0
patch_problem = 0

for fastq in fastq_files:
    fastq_status = fastq['status']
    if fastq_status in ['released', 'released to project', 'archived']:
        qc_status = ff_utils.get_metadata(fastq['quality_metric'],connection=ff)['status']
        if qc_status != fastq_status:
            patch_data = {"status": fastq_status}
            try:
                ff_utils.patch_metadata(patch_data, obj_id=fastq['quality_metric'] ,connection=ff)
                success += 1
                print success
            except Exception as e:
                print e
                print
                patch_problem += 1
print success
print patch_problem



2743
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


KeyboardInterrupt: 